# Single-Cell Report: BBKNN Batch Correction

In [ ]:
FILE1 = "/Users/u0125489/tmp/bbknn_testdata/BBKNN-test.SC__SCANPY__CLUSTERING.h5ad"
FILE2 = "/Users/u0125489/tmp/bbknn_testdata/BBKNN-test.SC__SCANPY__DIM_REDUCTION_UMAP.h5ad"

In [ ]:
# Import packages
import scanpy as sc
import matplotlib.pyplot as plt

#### Plotting settings and functions

In [ ]:
# plot settings
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)

#### Read Data

In [ ]:
adata1 = sc.read_h5ad(filename=FILE1)
adata2 = sc.read_h5ad(filename=FILE2)

---
## Batch effect correction

In [ ]:
fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(8,8), dpi=150 )
sc.pl.umap(adata1, color='batch', palette='Set2', ax=ax1, show=False)
sc.pl.umap(adata2, color='batch', palette='Set2', ax=ax2, show=False)
sc.pl.umap(adata1, color='louvain', palette=sc.pl.palettes.default_64, ax=ax3, show=False)
sc.pl.umap(adata2, color='louvain', palette=sc.pl.palettes.default_64, ax=ax4, show=False)

ax1.set_title('Pre-batch correction (batch)')
ax2.set_title('Post-batch correction (batch)')
ax3.set_title('Pre-batch correction (Louvain)')
ax4.set_title('Post-batch correction (Louvain)')
#
plt.tight_layout()